In [ ]:
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col,monotonically_increasing_id
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

In [ ]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [ ]:
create_spark_session()

In [ ]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://udacity-datalake-ouput/"

In [ ]:
song_data = input_data + 'song_data/*/*/*/*.json'

df = spark.read.json(song_data)
df.createOrReplaceTempView("song_data")

In [ ]:
songs_table = df['song_id', 'title', 'artist_id', 'year', 'duration'].where(col("song_id").isNotNull()).dropDuplicates(['song_id'])
songs_table.show(5)

In [ ]:
songs_table.write.mode('overwrite').partitionBy("year", "artist_id").parquet(output_data+'songs_table/')

In [ ]:
artists_table = df['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude'].dropDuplicates(['artist_id']).where(col("artist_id").isNotNull())
artists_table.show(5)

In [ ]:
artists_table.write.mode("overwrite").parquet(output_data+'artists_table/')


In [ ]:
log_data = input_data + 'log_data/*/*/*.json'
df = spark.read.json(log_data)
df.show(5)

In [ ]:
users_table = df['userId', 'firstName', 'lastName', 'gender', 'level'].dropDuplicates(['userId'])
users_table.show(5)

In [ ]:
users_table.write.mode('overwrite').parquet(output_data+'users_table/')

In [ ]:
get_timestamp = udf(lambda x: str(int(int(x)/1000)))
df = df.withColumn('timestamp', get_timestamp(df.ts))

get_datetime = udf(lambda x: str(datetime.fromtimestamp(int(x) / 1000.0)))
df = df.withColumn("datetime", get_datetime(df.ts))

In [ ]:
time_table = df.select(
        col('datetime').alias('start_time'),
        hour('datetime').alias('hour'),
        dayofmonth('datetime').alias('day'),
        weekofyear('datetime').alias('week'),
        month('datetime').alias('month'),
        year('datetime').alias('year') 
   )
time_table = time_table.dropDuplicates(['start_time'])

time_table.show(5)    

In [ ]:
time_table.write.mode('overwrite').partitionBy("year", "month").parquet(output_data+'time_table/')

In [ ]:
song_df = spark.read.json(song_data)
df = df.join(song_df, song_df.title == df.song)
df = df.withColumn("songplay_id", monotonically_increasing_id())

df.printSchema()
df.show(5)

In [ ]:
songplays_table = df.select(
        col('songplay_id').alias('songplay_id'),
        col('ts').alias('ts'),
        col('userId').alias('user_id'),
        col('level').alias('level'),
        col('song_id').alias('song_id'),
        col('artist_id').alias('artist_id'),
        col('sessionId').alias('session_id'),
        col('location').alias('location'),
        col('userAgent').alias('user_agent'),
        col('year').alias('year'),
        month('datetime').alias('month')
    )
    
songplays_table.show(5)